# Real-Time Clustering Inference Demo

This notebook demonstrates how the trained models work in real-time to cluster students during a live session.

**Scenario**: Simulate a student joining a session and answering questions, with cluster updates after each question.

**Prerequisites**: Run `01_Preprocessing_Final.ipynb` and `02_Model_Training_RealTime.ipynb` first.

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import numpy as np
import joblib
import pandas as pd

np.random.seed(42)

## Step 1: Load Trained Models

In [ ]:
models_dir = '/content/drive/MyDrive/FYP_Data/Models'

# Load models
kmeans = joblib.load(f'{models_dir}/kmeans_initial.pkl')
cluster_mapping = joblib.load(f'{models_dir}/cluster_mapping.pkl')
rf_model = joblib.load(f'{models_dir}/rf_dynamic.pkl')
scaler_initial = joblib.load(f'{models_dir}/scaler_initial.pkl')
scaler_dynamic = joblib.load(f'{models_dir}/scaler_dynamic.pkl')

print("✅ Models loaded successfully!")
print(f"\nCluster Mapping: {cluster_mapping}")

## Step 2: Create Real-Time Clustering Class

In [ ]:
class RealTimeClusteringSystem:
    def __init__(self, kmeans, cluster_mapping, rf_model, scaler_initial, scaler_dynamic):
        self.kmeans = kmeans
        self.cluster_mapping = cluster_mapping
        self.rf_model = rf_model
        self.scaler_initial = scaler_initial
        self.scaler_dynamic = scaler_dynamic
        self.student_state = {}
    
    def initial_cluster(self, student_id, response_time, rtt, jitter, stability):
        """
        Predict initial cluster when student first joins and answers initial question.
        """
        # Create feature vector
        features = np.array([[response_time, rtt, jitter, stability]])
        
        # Scale features
        features_scaled = self.scaler_initial.transform(features)
        
        # Predict cluster
        cluster_id = self.kmeans.predict(features_scaled)[0]
        cluster_label = self.cluster_mapping[cluster_id]
        
        # Initialize student state
        self.student_state[student_id] = {
            'correct': 0,
            'total': 0,
            'response_times': [],
            'cluster': cluster_label
        }
        
        return cluster_label
    
    def update_cluster(self, student_id, response_time, is_correct):
        """
        Update cluster after student answers a question.
        """
        if student_id not in self.student_state:
            raise ValueError(f"Student {student_id} not initialized. Call initial_cluster first.")
        
        state = self.student_state[student_id]
        
        # Update cumulative metrics
        state['correct'] += int(is_correct)
        state['total'] += 1
        state['response_times'].append(response_time)
        
        # Calculate features
        cumulative_accuracy = state['correct'] / state['total']
        avg_response_time = np.mean(state['response_times'])
        
        features = np.array([[
            cumulative_accuracy,
            avg_response_time,
            state['total'],
            response_time,
            int(is_correct)
        ]])
        
        # Scale features
        features_scaled = self.scaler_dynamic.transform(features)
        
        # Predict new cluster
        new_cluster = self.rf_model.predict(features_scaled)[0]
        
        # Check if cluster changed
        old_cluster = state['cluster']
        cluster_changed = (old_cluster != new_cluster)
        
        # Update state
        state['cluster'] = new_cluster
        
        return {
            'new_cluster': new_cluster,
            'old_cluster': old_cluster,
            'changed': cluster_changed,
            'accuracy': cumulative_accuracy,
            'avg_response_time': avg_response_time,
            'questions_answered': state['total']
        }
    
    def get_student_status(self, student_id):
        """Get current status of a student."""
        if student_id not in self.student_state:
            return None
        return self.student_state[student_id]

# Initialize system
clustering_system = RealTimeClusteringSystem(
    kmeans, cluster_mapping, rf_model, scaler_initial, scaler_dynamic
)

print("✅ Real-Time Clustering System initialized!")

## Step 3: Simulate Real-Time Session - Example 1

**Student Performance**: High performer (Active)

In [ ]:
print("="*60)
print("SIMULATION 1: High Performer")
print("="*60)

student_id = "2024001"

# Initial question
print("\n🔹 Student joins session and answers initial question")
print("   Response Time: 30s, Network: Good (RTT=100, Jitter=20, Stability=95%)")
initial_cluster = clustering_system.initial_cluster(student_id, 30, 100, 20, 95)
print(f"   ➡️ Initial Cluster: {initial_cluster}")

# Question 1
print("\n🔹 Question 1: Answered correctly in 20s")
result = clustering_system.update_cluster(student_id, 20, True)
print(f"   Accuracy: {result['accuracy']:.2%}, Avg Time: {result['avg_response_time']:.1f}s")
if result['changed']:
    print(f"   ➡️ Cluster Updated: {result['old_cluster']} → {result['new_cluster']} ✨")
else:
    print(f"   ➡️ Cluster: {result['new_cluster']} (no change)")

# Question 2
print("\n🔹 Question 2: Answered correctly in 18s")
result = clustering_system.update_cluster(student_id, 18, True)
print(f"   Accuracy: {result['accuracy']:.2%}, Avg Time: {result['avg_response_time']:.1f}s")
if result['changed']:
    print(f"   ➡️ Cluster Updated: {result['old_cluster']} → {result['new_cluster']} ✨")
else:
    print(f"   ➡️ Cluster: {result['new_cluster']} (no change)")

# Question 3
print("\n🔹 Question 3: Answered correctly in 22s")
result = clustering_system.update_cluster(student_id, 22, True)
print(f"   Accuracy: {result['accuracy']:.2%}, Avg Time: {result['avg_response_time']:.1f}s")
if result['changed']:
    print(f"   ➡️ Cluster Updated: {result['old_cluster']} → {result['new_cluster']} ✨")
else:
    print(f"   ➡️ Cluster: {result['new_cluster']} (no change)")

# Final status
status = clustering_system.get_student_status(student_id)
print("\n📊 Final Status:")
print(f"   Cluster: {status['cluster']}")
print(f"   Questions Answered: {status['total']}")
print(f"   Correct: {status['correct']}/{status['total']} ({status['correct']/status['total']:.1%})")
print(f"   Avg Response Time: {np.mean(status['response_times']):.1f}s")

## Step 4: Simulate Real-Time Session - Example 2

**Student Performance**: Declining performer (Active → Moderate → Passive)

In [ ]:
print("\n" + "="*60)
print("SIMULATION 2: Declining Performer")
print("="*60)

student_id = "2024002"

# Initial question
print("\n🔹 Student joins session and answers initial question")
print("   Response Time: 25s, Network: Good (RTT=90, Jitter=15, Stability=98%)")
initial_cluster = clustering_system.initial_cluster(student_id, 25, 90, 15, 98)
print(f"   ➡️ Initial Cluster: {initial_cluster}")

# Question 1 - Correct
print("\n🔹 Question 1: Answered correctly in 20s")
result = clustering_system.update_cluster(student_id, 20, True)
print(f"   Accuracy: {result['accuracy']:.2%}, Avg Time: {result['avg_response_time']:.1f}s")
if result['changed']:
    print(f"   ➡️ Cluster Updated: {result['old_cluster']} → {result['new_cluster']} ✨")
else:
    print(f"   ➡️ Cluster: {result['new_cluster']} (no change)")

# Question 2 - Incorrect
print("\n🔹 Question 2: Answered incorrectly in 45s")
result = clustering_system.update_cluster(student_id, 45, False)
print(f"   Accuracy: {result['accuracy']:.2%}, Avg Time: {result['avg_response_time']:.1f}s")
if result['changed']:
    print(f"   ➡️ Cluster Updated: {result['old_cluster']} → {result['new_cluster']} ⚠️")
else:
    print(f"   ➡️ Cluster: {result['new_cluster']} (no change)")

# Question 3 - Incorrect
print("\n🔹 Question 3: Answered incorrectly in 60s")
result = clustering_system.update_cluster(student_id, 60, False)
print(f"   Accuracy: {result['accuracy']:.2%}, Avg Time: {result['avg_response_time']:.1f}s")
if result['changed']:
    print(f"   ➡️ Cluster Updated: {result['old_cluster']} → {result['new_cluster']} ⚠️")
else:
    print(f"   ➡️ Cluster: {result['new_cluster']} (no change)")

# Question 4 - Incorrect
print("\n🔹 Question 4: Answered incorrectly in 70s")
result = clustering_system.update_cluster(student_id, 70, False)
print(f"   Accuracy: {result['accuracy']:.2%}, Avg Time: {result['avg_response_time']:.1f}s")
if result['changed']:
    print(f"   ➡️ Cluster Updated: {result['old_cluster']} → {result['new_cluster']} 🚨")
else:
    print(f"   ➡️ Cluster: {result['new_cluster']} (no change)")

# Final status
status = clustering_system.get_student_status(student_id)
print("\n📊 Final Status:")
print(f"   Cluster: {status['cluster']}")
print(f"   Questions Answered: {status['total']}")
print(f"   Correct: {status['correct']}/{status['total']} ({status['correct']/status['total']:.1%})")
print(f"   Avg Response Time: {np.mean(status['response_times']):.1f}s")
print("\n   🔔 Alert: Student needs attention!")

## Step 5: Batch Processing - Multiple Students

In [ ]:
print("\n" + "="*60)
print("BATCH PROCESSING: Multiple Students")
print("="*60)

# Simulate 5 students
students = [
    {"id": "2024003", "initial": (35, 120, 30, 90), "responses": [(25, True), (30, True), (28, False)]},
    {"id": "2024004", "initial": (50, 200, 80, 70), "responses": [(60, False), (65, False), (70, False)]},
    {"id": "2024005", "initial": (28, 100, 25, 95), "responses": [(20, True), (22, True), (19, True)]},
    {"id": "2024006", "initial": (40, 150, 50, 85), "responses": [(35, True), (40, False), (38, True)]},
    {"id": "2024007", "initial": (55, 180, 70, 75), "responses": [(50, False), (48, True), (52, False)]},
]

results_summary = []

for student in students:
    sid = student['id']
    
    # Initial clustering
    initial_cluster = clustering_system.initial_cluster(sid, *student['initial'])
    
    # Process responses
    for response_time, is_correct in student['responses']:
        result = clustering_system.update_cluster(sid, response_time, is_correct)
    
    # Get final status
    status = clustering_system.get_student_status(sid)
    
    results_summary.append({
        'Student ID': sid,
        'Initial Cluster': initial_cluster,
        'Final Cluster': status['cluster'],
        'Questions': status['total'],
        'Accuracy': f"{status['correct']/status['total']:.1%}",
        'Avg Time': f"{np.mean(status['response_times']):.1f}s"
    })

# Display summary
summary_df = pd.DataFrame(results_summary)
print("\n📊 Session Summary:")
print(summary_df.to_string(index=False))

print("\n📈 Cluster Distribution:")
print(summary_df['Final Cluster'].value_counts())

## Summary

**Real-Time Clustering System Demonstrated!**

Key Features:
- ✅ Initial clustering from first question
- ✅ Dynamic updates after each question
- ✅ Tracks cumulative performance
- ✅ Detects cluster transitions
- ✅ Supports multiple students simultaneously

**Production Deployment**:
1. Load trained models at server startup
2. When student joins: Call `initial_cluster()`
3. After each question: Call `update_cluster()`
4. Display current cluster to instructor in real-time
5. Alert instructor when student moves to Passive cluster

**No cluster history CSV needed** - Models predict clusters in real-time!